In [ ]:
pip install  langchain langchain-community pypdf langchain-huggingface sentence_transformers huggingface_hub langchain-text-splitters langchain-pinecone

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 1

In [ ]:
import os
import getpass
from google.colab import files
from pinecone import Pinecone, ServerlessSpec
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import uuid

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [ ]:
from google.colab import userdata


PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')



In [ ]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_dimension = 768
embeddings = HuggingFaceEndpointEmbeddings(
        model="sentence-transformers/all-mpnet-base-v2",
        huggingfacehub_api_token=HUGGINGFACE_API_KEY
    )

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

In [ ]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [ ]:
import time

index_name = "langchain-huggingface-index"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print(f"Deleted existing index: {index_name}")

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        deletion_protection="disabled",  
    )


index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
print(f"Creating index: {index_name}")


Creating index: langchain-huggingface-index


In [ ]:
uploaded = files.upload()

pdf_filename = list(uploaded.keys())[0]

loader = PyPDFLoader(pdf_filename)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
texts = text_splitter.split_documents(documents)


Saving Random Forest ML.pdf to Random Forest ML.pdf


In [ ]:
uuids = [str(uuid.uuid4()) for _ in range(len(texts))]
vector_store.add_documents(documents=texts, ids=uuids)
print(f"Successfully stored {len(texts)} document chunks in Pinecone!")

Successfully stored 26 document chunks in Pinecone!


In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="conversational",
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACE_API_KEY
)

chat_model = ChatHuggingFace(llm=llm)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

def ask_document_huggingface(question):
  """
  This function takes a user's question, retrieves relevant context,
  constructs a structured prompt with system and user messages,
  and returns the LLM's answer.
  """
  # 1. Retrieve relevant document chunks
  retriever = vector_store.as_retriever()
  retrieved_docs = retriever.invoke(question)

  context = "\n\n".join([doc.page_content for doc in retrieved_docs])

  messages = [
      SystemMessage(
          content=f"""
          You are a helpful assistant. Your task is to answer the user's question based only on the following context.
          If the answer is not available in the context, please say "I'm sorry, I don't have enough information to answer that question."

          Context:
          {context}
          """
      ),
      HumanMessage(content=question),
  ]


  result = chat_model.invoke(messages)
  return result.content

my_question = "What is the main topic of this document?"
answer = ask_document_huggingface(my_question)
print(f"Question: {my_question}")
print(f"Answer: {answer}")

Question: What is the main topic of this document?
Answer:  The main topic of this document appears to be Machine Learning Algorithms, specifically focusing on Bagging, Pasting, Random Forests, and Boosting.


In [ ]:
import os
import uuid
from dotenv import load_dotenv
from google.colab import files
from pinecone import Pinecone, ServerlessSpec
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_core.messages import HumanMessage, SystemMessage

embeddings = AzureOpenAIEmbeddings(
    azure_deployment='text-embedding-3-small',
    api_key=AZURE_OPENAI_ENDPOINT,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)


PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "langchain-azure-openai"

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print(f"Deleted existing index: {index_name}")

pc.create_index(
    name=index_name,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    deletion_protection="disabled",
    dimension=1536
)
print(f"Creating a new index: {index_name}")

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)


Creating a new index: langchain-azure-openai


In [ ]:
llm = AzureChatOpenAI(
    azure_deployment='gpt-4o',
    temperature=0.5,
    api_key=AZURE_OPENAI_KEY,
    api_version='2023-03-15-preview',
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)


In [ ]:

print("Please upload your PDF document.")
uploaded = files.upload()

pdf_filename = list(uploaded.keys())[0]

loader = PyPDFLoader(pdf_filename)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

uuids = [str(uuid.uuid4()) for _ in range(len(texts))]

vector_store.add_documents(documents=texts, ids=uuids)
print(f"Successfully stored {len(texts)} document chunks in Pinecone!")



Please upload your PDF document.


Saving Random Forest ML.pdf to Random Forest ML (1).pdf
Successfully stored 26 document chunks in Pinecone!


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [ ]:
def ask_document_with_azure(question: str) -> str:
    retriever = vector_store.as_retriever()
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    messages = [
        SystemMessage(
            content=(
                "You are a helpful assistant. Your task is to answer the user's question "
                "based only on the following context. If the answer is not in the context, "
                'say "I do not have enough information to answer that question."'
                f"\n\nContext:\n{context}"
            )
        ),
        HumanMessage(content=question),
    ]
    result = llm.invoke(messages)
    return result.content

my_question = "What is the main topic of this document?"
answer = ask_document_with_azure(my_question)
print("\n" + "="*50)
print(f"Question: {my_question}")
print(f"Answer: {answer}")
print("="*50 + "\n")


Question: What is the main topic of this document?
Answer: The main topic of this document is "Ensemble Learning and Random Forests," which discusses techniques for combining predictions from multiple predictors to improve accuracy and introduces methods like bagging, pasting, and the Random Forest algorithm.

